In [6]:
!pip install torchtext==0.17.0 --force-reinstall
!pip install transformers==4.38.2
!pip install accelerate==0.27.2

  Using cached torchtext-0.17.0-cp311-cp311-manylinux1_x86_64.whl.metadata (7.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.2.0-cp311-cp311-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached numpy-2.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached torchdata-0.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runti

^C


In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz

# **실습 코드를 기반으로 똑같이 LSTM 기반 학습 코드를 구현하여 10 epoch training에 대한 train / test 결과를 확인하세요. (20)**

In [1]:

import numpy as np
import torch
seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

import os
import pandas as pd
def load_imdb_split(split_dir):
    texts, labels = [], []
    for label in ['pos', 'neg']:
      folder = os.path.join(split_dir, label)
      for filename in os.listdir(folder):
        with open(os.path.join(folder, filename), 'r', encoding='utf-8') as f:
          texts.append(f.read())
          labels.append(1 if label == 'pos' else 0)
    return pd.DataFrame({'text': texts, 'label': labels})
train_df = load_imdb_split('aclImdb/train')
test_df = load_imdb_split('aclImdb/test')

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
def yield_tokens(text_series):
    for text in text_series:
       yield tokenizer(text)

# Build vocab using training data only
vocab = build_vocab_from_iterator(yield_tokens(train_df['text']), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

MAX_LEN=256
def encode_text(text):
    tokens = tokenizer(text)
    return vocab(tokens[:MAX_LEN])

train_df['input_ids'] = train_df['text'].apply(encode_text)
test_df['input_ids'] = test_df['text'].apply(encode_text)

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
def create_tensor_dataset(inputs, labels):
    tensor_seqs = [torch.tensor(x) for x in inputs]
    padded_seqs = pad_sequence(tensor_seqs, batch_first=True)
    labels_tensor = torch.tensor(list(labels), dtype=torch.long)

    return TensorDataset(padded_seqs, labels_tensor)

train_dataset = create_tensor_dataset(train_df['input_ids'], train_df['label'])
test_dataset = create_tensor_dataset(test_df['input_ids'], test_df['label'])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

import torch.nn as nn
class SimpleLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=100, hidden_dim=128, output_dim=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim,
        padding_idx=vocab["<unk>"])
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x) # (batch, seq_len, embedding_dim)
        output, (hidden, _) = self.lstm(embedded) # hidden: (1, batch, hidden_dim)
        hidden = hidden.squeeze(0) # (batch, hidden_dim)
        return self.sigmoid(self.fc(hidden))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleLSTMClassifier(vocab_size=len(vocab), embedding_dim = 300, hidden_dim =
128).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


from tqdm import tqdm
def train_loop(model, train_loader, test_loader, optimizer, criterion, device, epochs=5):
    model.to(device)
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        total_correct = 0
        print(f"\n Epoch {epoch}/{epochs}")
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")
        for batch_idx, (x_batch, y_batch) in progress_bar:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device).float()
            optimizer.zero_grad()
            outputs = model(x_batch).squeeze(1)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            predictions = (outputs >= 0.5).long()
            total_correct += (predictions == y_batch).sum().item()
            progress_bar.set_postfix(loss=loss.item())
        train_acc = total_correct / len(train_loader.dataset)
        avg_loss = total_loss / len(train_loader)
        model.eval()
        with torch.no_grad():
            correct = 0
            for x_batch, y_batch in test_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device).float()
                outputs = model(x_batch).squeeze(1)
                preds = (outputs >= 0.5).long()
                correct += (preds == y_batch).sum().item()
            test_acc = correct / len(test_loader.dataset)

        print(f"Epoch {epoch} Summary - Avg Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")
    return model

model = train_loop(
model=model,
train_loader=train_loader,
test_loader=test_loader,
optimizer=optimizer,
criterion=criterion,
device=device,
epochs=10
)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package


 Epoch 1/10


Training: 100%|██████████| 391/391 [00:09<00:00, 41.37it/s, loss=0.706]


Epoch 1 Summary - Avg Loss: 0.6932, Train Acc: 0.5064, Test Acc: 0.5038

 Epoch 2/10


Training: 100%|██████████| 391/391 [00:09<00:00, 43.04it/s, loss=0.676]


Epoch 2 Summary - Avg Loss: 0.6579, Train Acc: 0.6098, Test Acc: 0.5276

 Epoch 3/10


Training:  68%|██████▊   | 266/391 [00:05<00:02, 46.22it/s, loss=0.618]


KeyboardInterrupt: 

# **epoch, learning_rate, batch_size를 고정한채 LSTM 기반 모델의 test set 성능을 87% 이상으로 올려보고 어떻게 성능을 향상시켰는지 작성하세요. (20) +Glove (6B, 300dim) pretrained word embedding 모델을 적용하여 성능을 확인하세요. (20)**

In [ ]:
import torchtext
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleLSTMClassifier(vocab_size=len(vocab), embedding_dim = 300, hidden_dim =
128).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
vectors = torchtext.vocab.GloVe(name='6B', dim=300,cache='~/.vector_cache')
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
model.embedding.weight.data = pretrained_embedding


In [ ]:
model.embedding.weight.requires_grad = False

In [ ]:
import torch.nn as nn

def init_weights(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Embedding):
        nn.init.xavier_uniform_(m.weight)  # 예: Xavier 초기화
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                nn.init.zeros_(param.data)

# 모델에 초기화 적용
naive_transformer_model.apply(init_weights)


In [ ]:
model = train_loop(
model=model,
train_loader=train_loader,
test_loader=test_loader,
optimizer=optimizer,
criterion=criterion,
device=device,
epochs=10
)

# **실습코드를 활용하여 naïve_transformer_model을 training 하려면 되지 않는다. training 되도록 코드를 수정하세요. (20)**
**– training이 되도록 고치면 성능이 형편없다. epoch, learning_rate, batch_size를 고정한채 naïve_taransformer_model 기반 모델의 test set**
성능을 82% 이상으로 올려보고 어떻게 성능을 향상 시켰는지 작성하세요. (20)


In [ ]:
import torch
import torch.nn as nn
import math
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) /
    d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0) # (1, max_len, d_model)
    def forward(self, x):
       return x + self.pe[:, :x.size(1)].to(x.device)
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=300, num_heads=4, hidden_dim=128,
                 num_layers=2, num_classes=1, max_len=256, dropout=0.1, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.pos_encoder = PositionalEncoding(embedding_dim, max_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Linear(embedding_dim, 1)

    def forward(self, x):
        embedded = self.embedding(x)  # (batch, seq_len, emb_dim)
        encoded = self.pos_encoder(embedded)
        transformer_out = self.transformer_encoder(encoded)  # (batch, seq_len, emb_dim)
        pooled = transformer_out.mean(dim=1)  # mean pooling
        # return self.classifier(pooled).squeeze(1)  # (batch,)
        return torch.sigmoid(self.classifier(pooled))  # .squeeze(1) 제거, sigmoid 적용

naive_transformer_model = TransformerClassifier(
    vocab_size=len(vocab),
    embedding_dim=300,
    # num_heads=8,
    num_heads=4,
    hidden_dim=256,
    # num_layers=6,
    num_layers=2,
    pad_idx=vocab["<unk>"]
).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(naive_transformer_model.parameters(), lr=1e-3)

naive_transformer_model = train_loop(
    model=naive_transformer_model,
    train_loader=train_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=10
)

In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

In [ ]:


import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

import numpy as np
import torch
def load_imdb_split(split_dir):
    texts, labels = [], []
    for label in ['pos', 'neg']:
      folder = os.path.join(split_dir, label)
      for filename in os.listdir(folder):
        with open(os.path.join(folder, filename), 'r', encoding='utf-8') as f:
          texts.append(f.read())
          labels.append(1 if label == 'pos' else 0)
    return pd.DataFrame({'text': texts, 'label': labels})
train_df = load_imdb_split('aclImdb/train')
test_df = load_imdb_split('aclImdb/test')

train_df = load_imdb_split('aclImdb/train')
test_df = load_imdb_split('aclImdb/test')

# 2. BERT 토크나이저
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 3. Dataset 클래스 정의
class IMDBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        return input_ids, attention_mask, label

    def __len__(self):
        return len(self.texts)

# 4. DataLoader 준비
train_dataset = IMDBDataset(train_df['text'].tolist(), train_df['label'].tolist(), tokenizer)
test_dataset = IMDBDataset(test_df['text'].tolist(), test_df['label'].tolist(), tokenizer)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)
#다른 모델 "distilbert-base-uncased"
# 5. BERT 기반 분류 모델 정의
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state.mean(dim=1)
        logits = self.classifier(self.dropout(pooled))
        return logits.squeeze(1)

# 6. 학습 함수
def train_loop(model, train_loader, test_loader, optimizer, criterion, device, epochs=10):
    model.to(device)
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        total_correct = 0
        print(f"\nEpoch {epoch}/{epochs}")
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc="Training")

        for batch_idx, (input_ids, attention_mask, y_batch) in progress_bar:
            input_ids, attention_mask, y_batch = input_ids.to(device), attention_mask.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            predictions = (torch.sigmoid(outputs) >= 0.5).long()
            total_correct += (predictions == y_batch.long()).sum().item()
            progress_bar.set_postfix(loss=loss.item())

        train_acc = total_correct / len(train_loader.dataset)
        avg_loss = total_loss / len(train_loader)

        # 평가
        model.eval()
        with torch.no_grad():
            correct = 0
            for input_ids, attention_mask, y_batch in test_loader:
                input_ids, attention_mask, y_batch = input_ids.to(device), attention_mask.to(device), y_batch.to(device)
                outputs = model(input_ids, attention_mask)
                preds = (torch.sigmoid(outputs) >= 0.5).long()
                correct += (preds == y_batch.long()).sum().item()

            test_acc = correct / len(test_loader.dataset)

        print(f"Epoch {epoch} Summary - Avg Loss: {avg_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

    return model

# 7. 학습 실행
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BertClassifier(dropout=0.3)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    mymodel=train_loop(model, train_loader, test_loader, optimizer, criterion, device, epochs=3)
